# Analysing CIA Factbook Data Using SQL

### Introduction

The CIA Factbook is an interesting source of information, which provides demographic information about countries, like their population size, population growth, total land and water area and birth, death and migration rates. In this project we'll use the SQLite variant of the SQL language to analyze the CIA Factbook and, by doing so, to learn how SQL enables us to do so. We will be looking to return a couple of interesting statistics. First, we'll query and summarise some of the data from the CIA Factbook, then we will verify the data by exploring any potential outliers, and finally we will try to compute and find the most densely populated countries in the world.

### Querying the Data

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

In [4]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [5]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### Summary Statistics

In [27]:
%%sql
SELECT MIN(population) AS 'Minimum Population', MAX(population) AS 'Maximum Population', 
       MIN(population_growth) AS 'Minimum Population Growth', 
       MAX(population_growth) AS 'Maximum Population Growth'
  FROM facts;

Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


For this section we have explored the factbook and returned the minimum and maximum population for the countries in the dataset, as well as the minimum and maximum population growth. Here we observe a couple of remarkable numbers. There is a country with a population of zero, and there is also a country with more than 7.2 billion people. This requires further attention.

In [25]:
%%sql
SELECT *, MIN(population)
  FROM facts;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,MIN(population)
250,ay,Antarctica,None,280000,None,0,None,None,None,None,0


In [23]:
%%sql
SELECT *, MAX(population)
  FROM facts;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,MAX(population)
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None,7256490011


We have succesfully identified the 'countries' with the zero and 7.2 billion populations. As it turns out, the CIA Factbook includes a row for Antarctica, as well as one for the whole world. In order to compute the summary statistics in a more accurate and useful way, these two rows will be filtered out of the data that we want to use. This is done in the query below.

In [41]:
%%sql
SELECT MIN(population) AS min_population, MAX(population) AS max_population, 
       MIN(population_growth) AS min_pop_growth, 
       MAX(population_growth) AS max_pop_growth
  FROM facts
 WHERE name != 'World';

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


In [42]:
%%sql
SELECT AVG(population) AS avg_population, AVG(area) AS avg_area
  FROM facts;

Done.


avg_population,avg_area
62094928.32231405,555093.546184739


In the code above, the summary statistics were recomputed, this time excluding the row for the whole world. Now we can see that the most populated country in the dataset has about 1.3 billion people, instead of the 7.2 billion we observed earlier. We have also computed the average population size for all countries in the factbook, as well as the average total land area.

In [44]:
%%sql
SELECT name AS 'Densely Populated Countries'
  FROM facts
 WHERE population > (SELECT AVG(population) FROM facts) AND area < (SELECT AVG(area) FROM facts); 

Done.


Densely Populated Countries
Bangladesh
Germany
Japan
Philippines
Thailand
United Kingdom
Vietnam


### Conclusion

In this project we have queried data from the CIA Factbook. One outlier has been ignored, so that we found the most populated country to have 1.3 billion inhabitants. We then calculated the average population size and the average land area of all countries of the world, and analysed which countries are more populated than average and simultaneously smaller in terms of land area than average. We found that only seven countries match this pattern. These are Bangladesh, Germany, Japan, the Philippines, Thailand, the United Kingdom and Vietnam. 